In [1]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

In [2]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
import random
import os

In [4]:
# config
numEpochs = 50 # number of epochs to train the GPT+PT model
embeddingSize = 512 # the hidden dimension of the representation of both GPT and PT
numPoints=20 # number of points that we are going to receive to make a prediction about f given x and y, if you don't know then use the maximum
numVars=2 # the dimenstion of input points x, if you don't know then use the maximum
numYs=1 # the dimension of output points y = f(x), if you don't know then use the maximum
blockSize = 100 # spatial extent of the model for its context
batchSize = 128 # batch size of training data
dataDir = 'D:/Datasets/Symbolic Dataset/Datasets/MyDataGenerator'
dataInfo = 'XYE_{}Var_{}Points_{}EmbeddingSize'.format(numVars, numPoints, embeddingSize)
target = 'Skeleton' #'Skeleton' #'EQ'
dataFolder = "nv2_np[20, 21]_trR[-1.0, 4.0]_teR[4.1, 8.0]_templateBased"
addr = './SavedModels/bestModel/' # where to save model
maxNumFiles = 30
bestLoss = None # if there is any model to load as pre-trained one

In [5]:
import math
from torch.utils.data import Dataset

class CharDataset(Dataset):

    def __init__(self, data, block_size, chars, target='EQ'):
        data_size, vocab_size = len(data), len(chars)
        print('data has %d examples, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        
        # padding token
        self.paddingToken = '_'
        self.paddingID = self.stoi[self.paddingToken]
        self.stoi[self.paddingToken] = self.paddingID
        self.itos[self.paddingID] = self.paddingToken
        self.threshold = [-1,1]
        
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data # it should be a list of examples
        self.target = target
    
    def __len__(self):
        return len(self.data)-1

    def __getitem__(self, idx):
        # grab an example from the data
        chunk = self.data[idx] # sequence of tokens including x, y, eq, etc.
        
        try:
            chunk = json.loads(chunk) # convert the sequence tokens to a dictionary
        except:
            print("Couldn't convert to json: {}".format(chunk))
            
        # encode every character in the equation to an integer
        # < is SOS, > is EOS
        dix = [self.stoi[s] for s in '<'+chunk[self.target]+'>']
        inputs = dix[:-1]
        outputs = dix[1:]
        
        # add the padding to the equations
        paddingSize = max(self.block_size-len(inputs),0)
        paddingList = [self.paddingID]*paddingSize
        inputs += paddingList
        outputs += paddingList 
        
        # make sure it is not more than what should be
        inputs = inputs[:self.block_size]
        outputs = outputs[:self.block_size]
        
        # extract points from the input sequence
        points = torch.zeros(numVars+numYs, numPoints)
        for idx, xy in enumerate(zip(chunk['X'], chunk['Y'])):
            x = xy[0] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1]) == float or type(xy[1]) == int else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=self.threshold[1], 
                                 neginf=self.threshold[0])
            p[p>self.threshold[1]] = self.threshold[1] # clip the upper bound
            p[p<self.threshold[0]] = self.threshold[0] # clip the lower bound
            points[:,idx] = p
        
        """
        arrange data and targets so that the first i elements of x
        will be asked to predict the i-th element of y. Notice that
        the eventual language model will actually make block_size
        individual predictions at the same time based on this data,
        so we are being clever and amortizing the cost of the forward
        pass of the network. So for example if block_size is 4, then
        we could e.g. sample a chunk of text "hello", the integers in
        x will correspond to "hell" and in y will be "ello". This will
        then actually "multitask" 4 separate examples at the same time
        in the language model:
        - given just "h", please predict "e" as next
        - given "he" please predict "l" next
        - given "hel" predict "l" next
        - given "hell" predict "o" next
        
        In addition, because the DataLoader will create batches of examples,
        every forward/backward pass during traning will simultaneously train
        a LOT of predictions, amortizing a lot of computation. In particular,
        for a batched input of integers X (B, T) where B is batch size and
        T is block_size and Y (B, T), the network will during training be
        simultaneously training to make B*T predictions, all at once! Of course,
        at test time we can paralellize across batch B, but unlike during training
        we cannot parallelize across the time dimension T - we have to run
        a forward pass of the network to recover the next single character of the 
        sequence along each batch dimension, and repeatedly always feed in a next
        character to get the next one.
        
        So yes there is a big asymmetry between train/test time of autoregressive
        models. During training we can go B*T at a time with every forward pass,
        but during test time we can only go B at a time, T times, with T forward 
        passes.
        """
        inputs = torch.tensor(inputs, dtype=torch.long)
        outputs = torch.tensor(outputs, dtype=torch.long)
        return inputs, outputs, points


In [6]:
import json
from tqdm import tqdm
import glob
def processDataFiles(path, files):
    text = ''""
    for f in tqdm(files):
        with open(path+f, 'r') as h: 
            lines = h.read() # don't worry we won't run out of file handles
            if lines[-1]==-1:
                lines = lines[:-1]
            text += lines #json.loads(line)       
            
    return text

In [7]:
path = '{}/{}/Train/'.format(dataDir, dataFolder)
files = os.listdir(path)[:maxNumFiles]
text = processDataFiles(path, files)
chars = sorted(list(set(text))+['_','T','<','>']) # extract unique characters from the text before converting the text to a list
# T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
text = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(text) # shuffle the dataset, it's important for combined number of variables
train_dataset = CharDataset(text, blockSize, chars, target=target) 

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.40s/it]


data has 1000000 examples, 55 unique.


In [8]:
idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points = train_dataset.__getitem__(idx)
print('inputs:{}'.format(inputs))
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))
minimum, maximum = points.min(), points.max()
print(minimum, maximum)

inputs:tensor([21, 23,  6, 23,  5, 51, 11,  5,  5, 12, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35])
id:121958
inputs:<C+C*x1**2__________________________________________________________________________________________
outputs:C+C*x1**2>__________________________________________________________________________________________
points:tensor([[ 1.0000,  0.6286,  1.0000, -0.1924,  1.0000, -0.7695,  1.0000,  1.0000,
          1.0000,  1.0000, -0.2098,  1.0000,  1.0000,  1.0000,  0.9283,  1.0000,
          1.0000,  1.0000,  1.0000,  0.5266],
        [ 1.0000,  0.0149,  1.0000,  0.1280,  0.6555,  0.9165,  1.0000,  1.0000,
          0.2773,  0.

In [9]:
path = '{}/{}/Val/'.format(dataDir,dataFolder)
files = os.listdir(path)[:maxNumFiles]
textVal = processDataFiles(path, files)
textVal = textVal.split('\n') # convert the raw text to a set of examples
val_dataset = CharDataset(textVal, blockSize, chars, target=target)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.69it/s]

data has 1001 examples, 55 unique.


In [10]:
idx = np.random.randint(val_dataset.__len__())
inputs, outputs, points = val_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(-0.8418) tensor(1.)
id:435
inputs:<C+C*x2**4+C*x2**6__________________________________________________________________________________
outputs:C+C*x2**4+C*x2**6>__________________________________________________________________________________
points:tensor([[ 1.0000,  0.8979,  1.0000,  0.3671,  0.0885,  0.0163,  0.0645,  1.0000,
          1.0000,  1.0000, -0.8418,  0.2858,  1.0000,  1.0000,  1.0000,  0.7238,
          1.0000, -0.6291,  1.0000,  1.0000],
        [ 1.0000, -0.3883,  1.0000,  1.0000, -0.0074,  1.0000,  1.0000,  0.7143,
          0.5493,  1.0000,  0.5235,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  0.9995, -0.8131],
        [ 1.0000,  0.3849,  1.0000,  1.0000,  0.3986,  1.0000,  1.0000,  0.6338,
          0.4649,  1.0000,  0.4515,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000, -0.0518]])


In [11]:
path = '{}/{}/Test/'.format(dataDir,dataFolder)
files = os.listdir(path)[:maxNumFiles]
textTest = processDataFiles(path, files)
textTest = textTest.split('\n') # convert the raw text to a set of examples
# test_dataset_target = CharDataset(textTest, blockSize, chars, target=target)
test_dataset = CharDataset(textTest, blockSize, chars)

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 250.62it/s]

data has 1001 examples, 55 unique.


In [12]:
idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\ninputs:{}\noutputs:{}\npoints:{}'.format(idx,inputs,outputs,points))

tensor(-0.8976) tensor(1.)
id:860
inputs:<-0.4799+0.1539*x2+0.37*x2*x2_______________________________________________________________________
outputs:-0.4799+0.1539*x2+0.37*x2*x2>_______________________________________________________________________
points:tensor([[ 0.1395,  1.0000,  1.0000, -0.8976,  1.0000,  0.5761,  1.0000,  1.0000,
          1.0000,  1.0000,  1.0000, -0.5516,  1.0000,  1.0000, -0.1049,  0.6273,
          1.0000,  1.0000, -0.7850,  0.1188],
        [-0.5956, -0.4591,  1.0000,  1.0000,  1.0000,  0.6427,  1.0000,  0.9329,
          0.3172,  1.0000,  1.0000, -0.0686,  1.0000,  1.0000,  1.0000,  1.0000,
          1.0000,  1.0000, -0.3736,  0.6115],
        [-0.4403, -0.4726,  1.0000,  1.0000,  1.0000, -0.2282,  1.0000, -0.0143,
         -0.3939,  1.0000,  1.0000, -0.4887,  1.0000,  1.0000,  0.3746,  0.0799,
          1.0000,  1.0000, -0.4858, -0.2474]])


In [13]:
from mingpt.model import GPT, GPTConfig, PointNetConfig
pconf = PointNetConfig(embeddingSize=embeddingSize, 
                       numberofPoints=numPoints, 
                       numberofVars=numVars, 
                       numberofYs=numYs)
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=8, n_head=8, n_embd=embeddingSize, padding_idx=train_dataset.paddingID)
model = GPT(mconf, pconf)

05/26/2021 21:12:42 - INFO - mingpt.model -   number of parameters: 3.058740e+07


In [14]:
fName = '{}_SymbolicGPT_{}_{}_{}_MINIMIZE.txt'.format(dataInfo, 
                                             'GPT_PT_Summation', 
                                             'Padding',
                                             blockSize)

In [15]:
ckptPath = '{}/{}.pt'.format(addr,fName.split('.txt')[0])
print(ckptPath)
if bestLoss != None:
    # load the best model
    print('Model has been loaded!')
    model.load_state_dict(torch.load(ckptPath))

./SavedModels/bestModel//XYE_2Var_20Points_512EmbeddingSize_SymbolicGPT_GPT_PT_Summation_Padding_100_MINIMIZE.pt


In [ ]:
from mingpt.trainer import Trainer, TrainerConfig
import os

try: 
    os.mkdir(addr)
except:
    print('Folder already exists!')
    
# initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=numEpochs, batch_size=batchSize, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=2*len(train_dataset)*blockSize,
                      num_workers=0, ckpt_path=ckptPath)
trainer = Trainer(model, train_dataset, val_dataset, tconf, bestLoss)

try:
    trainer.train()
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Folder already exists!


epoch 1 iter 644: train loss 0.46551. lr 5.974868e-04:   8%|█▉                      | 645/7813 [05:00<54:49,  2.18it/s]

Input:tensor([21, 23,  6, 23,  5, 49, 41, 44,  3, 51, 11,  4,  5, 49, 41, 44,  3, 51,
        12,  4,  5,  5, 14,  6, 23,  5, 51, 11,  5,  5, 11,  6, 23,  5, 51, 12,
         5, 51, 11,  5,  5, 16,  6, 23,  5, 49, 47, 48, 50,  3, 51, 11,  4,  5,
        49, 47, 48, 50,  3, 51, 12,  4,  5,  5, 14,  6, 23,  5, 51, 11,  5, 51,
        11,  5,  5, 15, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35], device='cuda:0')
Logit:tensor([23,  6, 23,  5, 51, 47, 44,  3, 51, 11,  4,  5,  5, 41, 44,  3, 51, 11,
         4,  5,  5, 16,  6, 23,  5, 51, 11,  5,  5, 14,  6, 23,  5, 51, 11,  5,
         5, 11,  5,  5, 16,  6, 23,  5, 51, 47, 48, 50,  3, 51, 11,  4,  5,  5,
        47, 48, 50,  3, 51, 11,  4,  5,  5, 16, 22, 23,  5, 51, 11,  5,  5, 11,
         5,  5, 16, 22, 22,  5,  3,  3,  3,  3,  3,  3,  3,  3, 22,  3,  4,  3,
        11, 22,  3, 22,  3, 22,  3,  3, 22, 44], device='cuda:0')
Input:<C+C*sin(x1)*sin(x2)**4+C*x1**1+C*x2*x1**6+C*sqrt(

epoch 1 iter 1903: train loss 0.44602. lr 5.783001e-04:  24%|█████▎                | 1904/7813 [14:42<45:38,  2.16it/s]

Input:tensor([21, 23,  6, 23,  5, 51, 12,  6, 23,  5, 51, 12,  5, 51, 11,  5,  5, 11,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35], device='cuda:0')
Logit:tensor([23,  6, 23,  5, 51, 12,  5, 23,  5, 51, 12,  5,  5, 11,  5,  5, 14,  6,
        23,  5, 50, 50,  5, 23, 50,  5, 23,  5,  5,  5,  5, 23,  5, 23, 23, 23,
        23, 23,  5, 23, 12,  5, 23,  5, 23, 23, 23, 23,  5, 12, 12, 51, 23, 51,
        23, 23, 12, 22,  5, 11, 12, 23, 12, 51, 51, 23, 22, 51, 11, 11,  5, 23,
        11, 23, 11, 23, 12, 23, 23, 22, 23,  5,  5,  5, 51, 11, 22, 22, 23, 23,
        22, 22,  5, 11, 23,  5,  5, 23,  5, 23], device='cuda:0')
Input:<C+C*x2+C*x2*x1**1________________________________

...

epoch 1 iter 4338: train loss 0.43756. lr 4.929218e-04:  56%|████████████▏         | 4339/7813 [33:35<27:23,  2.11it/s]

Input:tensor([21, 23,  6, 23,  5, 51, 12,  5,  5, 14,  6, 23,  5, 51, 12,  5,  5, 49,
        47, 48, 50,  3, 51, 12,  4,  6, 23,  5, 49, 47, 48, 50,  3, 51, 11,  4,
         5, 49, 47, 48, 50,  3, 51, 11,  4,  5,  5, 11,  6, 23,  5, 51, 12,  5,
         5, 13,  6, 23,  5, 51, 11, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35], device='cuda:0')
Logit:tensor([23,  6, 23,  5, 51, 12,  5,  5, 11,  6, 23,  5, 51, 12,  5,  5, 11, 41,
        48, 50,  3, 51, 12,  4, 22, 23,  5, 51, 41, 48, 50,  3, 51, 12,  4,  5,
         5, 47, 48, 50,  3, 51, 12,  4,  5,  5, 11, 22, 23,  5, 51, 12,  5,  5,
        11, 22, 23,  5, 51, 12,  5, 12, 22, 12, 22, 23, 12, 12,  5, 12, 12, 12,
        12, 12, 12, 12, 22, 12, 12, 22, 12, 12, 12, 12, 22, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12,  3, 12, 12, 12, 12], device='cuda:0')
Input:<C+C*x2**4+C*x2**sqrt(x2)+C*sqrt(x1)*sqrt(x1)**1+C

epoch 1 iter 4367: train loss 0.43252. lr 4.915789e-04:  56%|████████████▎         | 4368/7813 [33:49<26:50,  2.14it/s]

Input:tensor([21, 23,  6, 23,  5, 51, 11,  6, 23,  5, 51, 12, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35,
        35, 35, 35, 35, 35, 35, 35, 35, 35, 35], device='cuda:0')
Logit:tensor([23,  6, 23,  5, 49, 12,  5, 23,  5, 51, 12, 22,  5, 22, 22, 22, 22, 22,
        22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 22, 23, 22, 22, 23, 22,
        22, 22, 23, 23, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 22, 22, 22,
        22, 22, 23, 22, 22, 22, 23, 23, 23, 22, 22, 22, 23, 22, 22, 23, 23, 22,
        23, 23, 22,  5, 23, 22, 22, 23, 23, 23, 22, 22, 22, 23,  5, 22, 22, 23,
        23, 22, 22, 22, 23, 51, 23, 22, 23, 22], device='cuda:0')
Input:<C+C*x1+C*x2______________________________________

epoch 1 iter 4548: train loss 0.42704. lr 4.830518e-04:  58%|████████████▊         | 4549/7813 [35:12<24:43,  2.20it/s]

In [ ]:
# load the best model
model.load_state_dict(torch.load(ckptPath))
model = model.eval().to(trainer.device)

In [ ]:
# add a safe wrapper for numpy math functions
from numpy import *
import numpy as np

def divide(x, y):
  x = np.nan_to_num(x)
  y = np.nan_to_num(y)
  return np.divide(x,y+1e-5)

def sqrt(x):
  x = np.nan_to_num(x)
  return np.sqrt(np.abs(x)) 

# Mean square error
def mse(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        our_sum += (y_hat[i] - y_gold[i]) ** 2

    return our_sum / len(y_gold)

# Mean square error
def relativeErr(y, y_hat):
    y_hat = np.reshape(y_hat, [1, -1])[0]
    y_gold = np.reshape(y, [1, -1])[0]
    our_sum = 0
    for i in range(len(y_gold)):
        if y_gold[i] < 1: 
            # use regular MSE
            our_sum += (y_hat[i] - y_gold[i]) ** 2
        else:
            # use relative MSE
            our_sum += ((y_hat[i] - y_gold[i])/y_gold[i]) ** 2

    return our_sum / len(y_gold)

In [ ]:
# class pointsDataset(Dataset):

#     def __init__(self, data):
#         # data should be a list of x,y pairs
#         self.x = data[0] # it should be a list
#         self.y = data[1] # it should be a list
#         self.threshold = [-1000,1000]
    
#     def __len__(self):
#         return len(self.y)

#     def __getitem__(self, idx):
#         # grab an example from the data
#         x = self.x[idx] 
#         y = self.y[idx] 
        
#         p = x+[y]
        
#         p = torch.tensor(p, dtype=torch.float)
#         p = torch.nan_to_num(p, nan=0.0, 
#                              posinf=self.threshold[1], 
#                              neginf=self.threshold[0])
#         p[p>self.threshold[1]] = self.threshold[1] # clip the upper bound
#         p[p<self.threshold[0]] = self.threshold[0] # clip the lower bound
        
#         return p
    
# # train a mlp to find the constants
# data = pointsDataset((t['X'],t['Y']))
# loader = torch.utils.data.DataLoader(
#                                 data, 
#                                 shuffle=False, 
#                                 pin_memory=True,
#                                 batch_size=batchSize,
#                                 num_workers=0)
# class MLP(nn.Module):
#     def __init__(self, inputSize, outputSize):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(inputSize, 100),
#             nn.ReLU(),
#             nn.Linear(100, outputSize)
#         )

#     def forward(self, x):
#         c = self.layers(x)
#         return c
    
#     def loss(self, constants, eq, X, Y):
#         # constants is the output of the model
#         err = torch.zeros(len(constants)) # batch_size
        
#         # sample number of points
#         indexes = []
#         numberSamples = 10
#         while len(indexes) != numberSamples:
#             randNum = np.random.randint(len(batch))
#             indexes.append(randNum)
#         X = X[indexes,:]
#         Y = Y[indexes,:]
        
#         # replace the constants with their predicted values
#         for idx, const in enumerate(constants):
#             eq = eq.replace('C','{}').format(*const.tolist())
        
#         # calculate the error for a limited number of points, approximate the error
#         for x,y in zip(X,Y):
#             # replace variables with their values
#             for i,e in enumerate(x):
#                 eqTemp = eq.replace('x{}'.format(i+1), str(e.item()))
                
#             # calculate the error
#             yHat = eval(eqTemp)
#             err[idx] += (y.item()-yHat)**2
#         err[idx] /= numberSamples
            
#         print(err.shape, constants.shape)
#         return err
    
# c = [0 for i,x in enumerate(predicted) if x=='C']
# network = MLP(numVars+numYs, len(c))
# cHat = network(batch)
# err = network.loss(cHat, predicted, batch[:,:numVars], batch[:,-numYs:])

In [ ]:
# alright, let's sample some character-level symbolic GPT
from mingpt.utils import sample
from gp_model import Genetic_Model
from mlp_model import MLP_Model
from scipy.optimize import least_squares, newton, minimize

    
loader = torch.utils.data.DataLoader(
                                test_dataset, 
                                shuffle=False, 
                                pin_memory=True,
                                batch_size=1,
                                num_workers=0)

testRange = [3.1,6.0]
numTestPoints = 10
#test = np.linspace(3.1,6.0,numTestPoints)

gpm = Genetic_Model(n_jobs=-1)
mlp = MLP_Model()

resultDict = {}
try:
    with open(fName, 'w', encoding="utf-8") as o:
        modelName = 'SymbolicGPT'
        resultDict[fName] = {modelName:[],
                             'GP':[],
                             'MLP':[]}

        for i, batch in enumerate(loader):
                
            inputs,outputs,points = batch

            print('Test Case {}.'.format(i))
            o.write('Test Case {}/{}.\n'.format(i,len(textTest)-1))

            t = json.loads(textTest[i])

            inputs = inputs[:,0:1].to(trainer.device)
            points = points.to(trainer.device)
            outputsHat = sample(model, inputs, blockSize, points=points,
                          temperature=1.0, sample=True, 
                          top_k=40)[0]

            # filter out predicted
            target = ''.join([train_dataset.itos[int(i)] for i in outputs[0]])
            predicted = ''.join([train_dataset.itos[int(i)] for i in outputsHat])

            target = target.strip(train_dataset.paddingToken).split('>')
            target = target[0] if len(target[0])>=1 else target[1]
            target = target.strip('<').strip(">")
            predicted = predicted.strip(train_dataset.paddingToken).split('>')
            predicted = predicted[0] if len(predicted[0])>=1 else predicted[1]
            predicted = predicted.strip('<').strip(">")
            
            print('Target:{}\nSkeleton:{}'.format(target, predicted))
            
            o.write('{}\n'.format(target))
            o.write('{}:\n'.format(modelName))
            o.write('{}\n'.format(predicted))

            # train a regressor to find the constants (too slow)
            c = [1 for i,x in enumerate(predicted) if x=='C']            
            def lossFunc(constants, eq, X, Y):
                err = 0
                eq = eq.replace('C','{}').format(*constants)

                for x,y in zip(X,Y):
                    eqTemp = eq + ''
                    for i,e in enumerate(x):
                        eqTemp = eqTemp.replace('x{}'.format(i+1), str(e))
                    try:
                        yHat = eval(eqTemp)
                    except:
                        yHat = 100
                    err += (y-yHat)**2
                err /= len(Y)
                return err
            
            try:
                if len(c) == 0:
                    pass # do nothing
                else:
#                     cHat = least_squares(lossFunc, c, ftol=1e-3,
#                                          args=(predicted, t['X'], t['Y']))
                    
                    # for easier comparison, we are using minimize package  
                    cHat = minimize(lossFunc, c,
                                   args=(predicted, t['X'], t['Y'])) 
            
#                     cHat= newton(lossFunc, c, maxiter=100,
#                                  args=(predicted, t['X'], t['Y']))
                    predicted = predicted.replace('C','{}').format(*cHat.x)
            except:
                print('Wrong Equation:{}'.format(predicted))
                raise
                predicted = 0

            # TODO: let's enjoy GPU

            print('Skeleton+LS:{}'.format(predicted))

            Ys = [] #t['YT']
            Yhats = []
            for xs in t['XT']:
                try:
                    eqTmp = target + '' # copy eq
                    eqTmp = eqTmp.replace(' ','')
                    eqTmp = eqTmp.replace('\n','')
                    for i,x in enumerate(xs):
                        # replace xi with the value in the eq
                        eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                        if ',' in eqTmp:
                            assert 'There is a , in the equation!'
                    YEval = eval(eqTmp)
                    YEval = 0 if np.isnan(YEval) else YEval
                    YEval = 100 if np.isinf(YEval) else YEval
                except:
                    YEval = 100 #TODO: Maybe I have to punish the model for each wrong template not for each point
                Ys.append(YEval)
                try:
                    eqTmp = predicted + '' # copy eq
                    eqTmp = eqTmp.replace(' ','')
                    eqTmp = eqTmp.replace('\n','')
                    for i,x in enumerate(xs):
                        # replace xi with the value in the eq
                        eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                        if ',' in eqTmp:
                            assert 'There is a , in the equation!'
                    Yhat = eval(eqTmp)
                    Yhat = 0 if np.isnan(Yhat) else Yhat
                    Yhat = 100 if np.isinf(Yhat) else Yhat
                except:
                    Yhat = 100
                Yhats.append(Yhat)
            err = relativeErr(Ys,Yhats)

            if type(err) is np.complex128 or np.complex:
                err = abs(err.real)

            resultDict[fName][modelName].append(err)

            o.write('{}\n{}\n\n'.format( 
                                    predicted,
                                    err
                                    ))

            print('Err:{}'.format(err))
            
            # Calculate error for baselines

    #         # tokenize to get input x, input y, and true eqn
    #         train_data_x = t["X"]
    #         train_data_y = t["Y"]
    #         test_data_x = t["XT"]
    #         test_data_y = t["YT"]

    #         # train MLP model
    #         mlp.reset()
    #         model_eqn, _, best_err = mlp.repeat_train(
    #                                                 train_data_x, 
    #                                                 train_data_y,
    #                                                 test_x=test_data_x, 
    #                                                 test_y=test_data_y,                                     
    #                                                 verbose=False)
    #         test_y_hat = mlp.predict(test_data_x)
    #         err = relativeErr(test_data_y,test_y_hat)
    #         print("{}: {}".format(mlp.name, model_eqn)[:550])
    #         print("Err: {:.5f}".format(err))
    #         resultDict[fName]['MLP'].append(err)
    #         o.write('\n{}: {}\n{}'.format('MLP', 
    #                                    err,
    #                                    model_eqn))

    #         # train GP model
    #         gpm.reset()
    #         model_eqn, _, best_err = gpm.repeat_train(train_data_x, train_data_y,
    #                                                 test_x=test_data_x, test_y=test_data_y,
    #                                                 verbose=False)
    #         print("{}: {}".format(gpm.name, model_eqn)[:550])
    #         test_y_hat = gpm.predict(test_data_x)
    #         err = relativeErr(test_data_y,test_y_hat)
    #         print("Err: {:.5f}".format(err))
    #         resultDict[fName]['GP'].append(err)
    #         o.write('\n{}: {}\n{}'.format('GP', 
    #                                    err,
    #                                    model_eqn))
            print('') # just an empty line
    print('Avg Err:{}'.format(np.mean(resultDict[fName][modelName])))
    
except KeyboardInterrupt:
    print('KeyboardInterrupt')

In [ ]:
# plot the error frequency for model comparison
from matplotlib import pyplot as plt
num_eqns = len(resultDict[fName]['SymbolicGPT'])
num_vars = pconf.numberofVars

models = list(key for key in resultDict[fName].keys() if len(resultDict[fName][key])==num_eqns)
lists_of_error_scores = [resultDict[fName][key] for key in models if len(resultDict[fName][key])==num_eqns]
linestyles = ["-","dashdot","dotted","--"]

eps = 0.00001
y, x, _ = plt.hist([np.log([max(min(x+eps, 1e5),1e-5) for x in e]) for e in lists_of_error_scores],
                   label=models,
                   cumulative=True, 
                   histtype="step", 
                   bins=2000, 
                   density=True,
                   log=False)
y = np.expand_dims(y,0)
plt.figure(figsize=(15, 10))

for idx, m in enumerate(models): 
    plt.plot(x[:-1], 
           y[idx] * 100, 
           linestyle=linestyles[idx], 
           label=m)

plt.legend(loc="upper left")
plt.title("{} equations of {} variables - Benchmark".format(num_eqns, num_vars))
plt.xlabel("Log of Relative Mean Square Error")
plt.ylabel("Normalized Cumulative Frequency")

name = '{}.png'.format(fName.split('.txt')[0])
print(name)
plt.savefig(name)

In [ ]:
# benchmarks
import csv
benchmarkPath = './benchmark/dsr-benchmark-data/'
dataPoints = glob.glob(benchmarkPath+'*.csv')
NGUYEN2Eq = {
           '1':'x1**3+x1**2+x1',
           '2':'x1**4+x1**3+x1**2+x1',
           '3':'x1**5+x1**4+x1**3+x1**2+x1',
           '4':'x1**6+x1**5+x1**4+x1**3+x1**2+x1',
           '5':'sin(x1**2)*cos(x1)-1',
           '6':'sin(x1)+sin(x1+x1**2)',
           '7':'log(x1+1)+log(x1**2+1)',
           '8':'sqrt(x1)',
           '9':'sin(x1)+sin(x2**2)',
           '10':'2*sin(x1)*cos(x2)',
           '11':'x1**x2',
           '12':'x1**4-x1**3+x2**2/2-x2',}
for dataPoint in dataPoints: 
    key = dataPoint.split('\\')[-1].split('Nguyen-')[-1].split('_')[0]
    if not key in NGUYEN2Eq.keys():
        continue
    target = NGUYEN2Eq[key]
    with open(dataPoint, newline='\n') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='|')
        pointsList = []
        pointsListTest = []
        for i, c in enumerate(reader):
            p = ([eval(x) for x in c[:-1]],eval(c[-1]))
            if i < numPoints:
                pointsList.append(p)
            else:
                pointsListTest.append(p)
                
        #pointsList = [([eval(x) for x in p[:-1]],eval(p[-1])) for i,p in enumerate(reader) if i < numPoints]
        #pointsListTest = [([eval(x) for x in p[:-1]],eval(p[-1])) for i,p in enumerate(reader) if i >= numPoints]
        
        # initialized the input variable with start token <
        inputs = torch.tensor([[train_dataset.stoi['<']]]).to(trainer.device)
        
        # extract points from the input sequence
        points = torch.zeros(numVars+numYs, numPoints)
        for idx, xy in enumerate(pointsList):
            x = xy[0][:numVars] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1])== float else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=train_dataset.threshold[1], 
                                 neginf=train_dataset.threshold[0])
            p[p>train_dataset.threshold[1]] = train_dataset.threshold[1] # clip the upper bound
            p[p<train_dataset.threshold[0]] = train_dataset.threshold[0] # clip the lower bound
            points[:,idx] = p
            
        points = points.unsqueeze(0).to(trainer.device)
        print(points)
        outputsHat = sample(model, inputs, blockSize, points=points,
                      temperature=1.0, sample=True, 
                      top_k=40)[0]
        
        # filter out predicted
        predicted = ''.join([train_dataset.itos[int(i)] for i in outputsHat])
        predicted = predicted.strip(train_dataset.paddingToken).split('>')
        predicted = predicted[0] if len(predicted[0])>=1 else predicted[1]
        predicted = predicted.strip('<').strip(">")
        
        # extract points from the input sequence
        pointsTest = torch.zeros(numVars+numYs, numPoints).numpy()
        for idx, xy in enumerate(pointsListTest):
            x = xy[0][:numVars] + [0]*(max(numVars-len(xy[0]),0)) # padding
            y = [xy[1]] if type(xy[1])== float else xy[1]
            y = y + [0]*(max(numYs-len(y),0)) # padding
            p = x+y # because it is only one point 
            p = torch.tensor(p)
            #replace nan and inf
            p = torch.nan_to_num(p, nan=0.0, 
                                 posinf=train_dataset.threshold[1], 
                                 neginf=train_dataset.threshold[0])
            p[p>train_dataset.threshold[1]] = train_dataset.threshold[1] # clip the upper bound
            p[p<train_dataset.threshold[0]] = train_dataset.threshold[0] # clip the lower bound
            pointsTest[:,idx] = p
        
        Ys = [] 
        Yhats = []
        for xs in pointsTest[:-1,:].T:
            try:
                eqTmp = target + '' # copy eq
                eqTmp = eqTmp.replace(' ','')
                eqTmp = eqTmp.replace('\n','')
                for i,x in enumerate(xs):
                    # replace xi with the value in the eq
                    eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                    if ',' in eqTmp:
                        assert 'There is a , in the equation!'
                YEval = eval(eqTmp)
                YEval = 0 if np.isnan(YEval) else YEval
                YEval = 100 if np.isinf(YEval) else YEval
            except:
                YEval = 100 #TODO: Maybe I have to punish the model for each wrong template not for each point
            Ys.append(YEval)
            try:
                eqTmp = predicted + '' # copy eq
                eqTmp = eqTmp.replace(' ','')
                eqTmp = eqTmp.replace('\n','')
                for i,x in enumerate(xs):
                    # replace xi with the value in the eq
                    eqTmp = eqTmp.replace('x{}'.format(i+1), str(x))
                    if ',' in eqTmp:
                        assert 'There is a , in the equation!'
                Yhat = eval(eqTmp)
                Yhat = 0 if np.isnan(Yhat) else Yhat
                Yhat = 1000 if np.isinf(Yhat) else Yhat
            except:
                Yhat = 1000
            Yhats.append(Yhat)
        err = relativeErr(Ys,Yhats)
        
        print('NGUYEN-{} --> Target:{}\nPredicted:{}\nErr:{}\n'.format(key, target, predicted, err))